In [1]:
!pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 44.4 MB/s eta 0:00:00


In [10]:
import numpy as np
import pandas as pd
import rdkit
from rdkit import Chem, rdBase, DataStructs
from rdkit.Chem import rdmolfiles, rdmolops, AllChem, Draw, rdDepictor, Descriptors
from rdkit.Chem.Draw import IPythonConsole, rdMolDraw2D
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem.Fingerprints import FingerprintMols

from IPython.display import SVG
from IPython.display import HTML
from IPython.display import display

from collections import defaultdict
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('./drive/MyDrive/AI/Material Informatics (MI)/Ring Repeating Units/Table of Supporting Information(SMILES)- Cal band gap_(RU1, RU2, RRU).csv')

In [5]:
df.head()

,NO.,Repeating units_RU1,Repeating units_RU2,SMILES_RU1,SMILES_RU2,SMILES_RRU,DFT band gap/eV
0,1,CH2-CO-NH-CS,CO-NH-CS-CH2,*-CC(=O)NC(-*)=S,*-CC(=S)NC(-*)=O,O=C1CC(=S)N1,2.691
1,2,CH2-CS-C6H4-O,CS-C6H4-O-CH2,*-CC(=S)C1=CC=C(O-*)C=C1,*-COC1=CC=C(C=C1)C(-*)=S,S=C1COC2=CC=C1C=C2,2.041
2,3,C6H4-C6H4-C4H2S-CS,C6H4-C4H2S-CS-C6H4,*-C(=S)C1=CC=C(S1)C1=CC=C(C=C1)C1=CC=C(-*)C=C1,*-C1=CC=C(C=C1)C(=S)C1=CC=C(S1)C1=CC=C(-*)C=C1,S=C1C2=CC=C(S2)C2=CC=C(C=C2)C2=CC=C1C=C2,1.807
3,4,CO-O-C4H2S-CS,O-C4H2S-CS-CO,*-C(=O)OC1=CC=C(S1)C(-*)=S,*-OC1=CC=C(S1)C(=S)C(-*)=O,O=C1OC2=CC=C(S2)C1=S,1.918
4,5,C6H4-CS-C4H2S-O,CS-C4H2S-O-C6H4,*-OC1=CC=C(S1)C(=S)C1=CC=C(-*)C=C1,*-C(=S)C1=CC=C(OC2=CC=C(-*)C=C2)S1,S=C1C2=CC=C(OC3=CC=C1C=C3)S2,1.930


In [6]:
df["mol_ru1"] = df["SMILES_RU1"].apply(lambda x:Chem.MolFromSmiles(x))
df["mol_ru2"] = df["SMILES_RU2"].apply(lambda x:Chem.MolFromSmiles(x))
df["mol_rru"] = df["SMILES_RRU"].apply(lambda x:Chem.MolFromSmiles(x))

In [7]:
df.head()

,NO.,Repeating units_RU1,Repeating units_RU2,SMILES_RU1,SMILES_RU2,SMILES_RRU,DFT band gap/eV,mol_ru1,mol_ru2,mol_rru
0,1,CH2-CO-NH-CS,CO-NH-CS-CH2,*-CC(=O)NC(-*)=S,*-CC(=S)NC(-*)=O,O=C1CC(=S)N1,2.691,<rdkit.Chem.rdchem.Mol object at 0x7f0df90c05f0>,<rdkit.Chem.rdchem.Mol object at 0x7f0df881aeb0>,<rdkit.Chem.rdchem.Mol object at 0x7f0df8826cf0>
1,2,CH2-CS-C6H4-O,CS-C6H4-O-CH2,*-CC(=S)C1=CC=C(O-*)C=C1,*-COC1=CC=C(C=C1)C(-*)=S,S=C1COC2=CC=C1C=C2,2.041,<rdkit.Chem.rdchem.Mol object at 0x7f0e30672f90>,<rdkit.Chem.rdchem.Mol object at 0x7f0df881af90>,<rdkit.Chem.rdchem.Mol object at 0x7f0df8826dd0>
2,3,C6H4-C6H4-C4H2S-CS,C6H4-C4H2S-CS-C6H4,*-C(=S)C1=CC=C(S1)C1=CC=C(C=C1)C1=CC=C(-*)C=C1,*-C1=CC=C(C=C1)C(=S)C1=CC=C(S1)C1=CC=C(-*)C=C1,S=C1C2=CC=C(S2)C2=CC=C(C=C2)C2=CC=C1C=C2,1.807,<rdkit.Chem.rdchem.Mol object at 0x7f0dfcfc30b0>,<rdkit.Chem.rdchem.Mol object at 0x7f0df881af20>,<rdkit.Chem.rdchem.Mol object at 0x7f0df8826d60>
3,4,CO-O-C4H2S-CS,O-C4H2S-CS-CO,*-C(=O)OC1=CC=C(S1)C(-*)=S,*-OC1=CC=C(S1)C(=S)C(-*)=O,O=C1OC2=CC=C(S2)C1=S,1.918,<rdkit.Chem.rdchem.Mol object at 0x7f0e2f16b820>,<rdkit.Chem.rdchem.Mol object at 0x7f0df881f040>,<rdkit.Chem.rdchem.Mol object at 0x7f0df8826e40>
4,5,C6H4-CS-C4H2S-O,CS-C4H2S-O-C6H4,*-OC1=CC=C(S1)C(=S)C1=CC=C(-*)C=C1,*-C(=S)C1=CC=C(OC2=CC=C(-*)C=C2)S1,S=C1C2=CC=C(OC3=CC=C1C=C3)S2,1.930,<rdkit.Chem.rdchem.Mol object at 0x7f0df8883200>,<rdkit.Chem.rdchem.Mol object at 0x7f0df881f0b0>,<rdkit.Chem.rdchem.Mol object at 0x7f0df8826eb0>


In [11]:
class Morgan_fingerprint(): #From Chen

    def __init__(self,mols):
        self.mols = mols
        self.bitExamples = {}
        self.keepMols = {}
        return
    
    def calculate(self,radius=2,bits=1024,vector_type="count",unhash=False):
        if vector_type not in ["count","bit"]: 
            return
        elif vector_type=="bit":
            unhash=False
            fp_all = defaultdict()
            
        mols = self.mols
        self.bits = bits
        self.radius = radius
        self.bitI_morgan =  {}
        morgan_count = defaultdict(lambda:[0]*len(mols))
        for idx,mol in enumerate(tqdm(mols)):
            self.bitI_morgan[idx] = {}
            if unhash==True:
                fp_morgan = AllChem.GetMorganFingerprint(mol, radius, bitInfo=self.bitI_morgan[idx])
            else:
                fp_morgan = AllChem.GetMorganFingerprintAsBitVect(mol, radius, 
                                                                  bitInfo=self.bitI_morgan[idx],nBits=bits)
                if vector_type=="bit":
                    arr = np.zeros((1,))
                    DataStructs.ConvertToNumpyArray(fp_morgan, arr)
                    fp_all[idx]= arr
            for key in list(self.bitI_morgan[idx].keys()):
                morgan_count["{}".format(key)][idx]=len(self.bitI_morgan[idx][key])
                #generate bit examples for visualization
                self.bitExamples[key] = idx
                self.keepMols[idx]=mol
                
        if vector_type=="bit": return pd.DataFrame.from_dict(fp_all).T #Bit type       
        return pd.DataFrame.from_dict(morgan_count)

    def visualize(self,bit_name,molSize=(250,200),verbose=True):
        bitId = int(bit_name)
        if verbose==True: print ('bit %s'%bitId)
        zid = self.bitExamples[bitId]
        mol = self.keepMols[zid]
        aid,rad = self.bitI_morgan[zid][bitId][0]
        pic1,svg  = getSubstructDepiction(mol,aid,rad,molSize=molSize)
        pic2 = Draw.DrawMorganBit(mol,bitId,self.bitI_morgan[zid])
        #display(pic)
        return pic1,pic2,svg

#for visualization #From Chen
def _prepareMol(mol,kekulize):
    mc = Chem.Mol(mol.ToBinary())
    if kekulize:
        try:
            Chem.Kekulize(mc)
        except:
            mc = Chem.Mol(mol.ToBinary())
    if not mc.GetNumConformers():
        rdDepictor.Compute2DCoords(mc)
    return mc

def moltosvg(mol,molSize=(450,200),kekulize=True,drawer=None,**kwargs):
    mc = _prepareMol(mol,kekulize)
    if drawer is None:
        drawer = rdMolDraw2D.MolDraw2DSVG(molSize[0],molSize[1])
    drawer.DrawMolecule(mc,**kwargs)
    drawer.FinishDrawing()
    svg = drawer.GetDrawingText()
    # It seems that the svg renderer used doesn't quite hit the spec.
    # Here are some fixes to make it work in the notebook, although I think
    # the underlying issue needs to be resolved at the generation step
    #with open('morgan bits/%s.svg'%bitId, 'w') as f:
    #    f.write(svg)
    return SVG(svg.replace('svg:','')),svg 

def getSubstructDepiction(mol,atomID,radius,molSize=(450,200)):
    if radius>0:
        env = Chem.FindAtomEnvironmentOfRadiusN(mol,radius,atomID)
        atomsToUse=[]
        for b in env:
            atomsToUse.append(mol.GetBondWithIdx(b).GetBeginAtomIdx())
            atomsToUse.append(mol.GetBondWithIdx(b).GetEndAtomIdx())
        atomsToUse = list(set(atomsToUse))       
    else:
        atomsToUse = [atomID]
        env=None
    return moltosvg(mol,molSize=molSize,highlightAtoms=atomsToUse)

In [12]:
mf_ru1 = Morgan_fingerprint(df["mol_ru1"].values)
fp_ru1 = mf_ru1.calculate(radius=2,bits=1024,vector_type="count",unhash=False)
fp_ru1 = fp_ru1.dropna(1)

mf_ru2 = Morgan_fingerprint(df["mol_ru2"].values)
fp_ru2 = mf_ru2.calculate(radius=2,bits=1024,vector_type="count",unhash=False)
fp_ru2 = fp_ru2.dropna(1)

mf_rru = Morgan_fingerprint(df["mol_rru"].values)
fp_rru = mf_rru.calculate(radius=2,bits=1024,vector_type="count",unhash=False)
fp_rru = fp_rru.dropna(1)

100%|██████████| 284/284 [00:00<00:00, 8600.53it/s]
<ipython-input-12-6c302e787d8f>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  fp_ru1 = fp_ru1.dropna(1)
100%|██████████| 284/284 [00:00<00:00, 9049.54it/s]
<ipython-input-12-6c302e787d8f>:7: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  fp_ru2 = fp_ru2.dropna(1)
100%|██████████| 284/284 [00:00<00:00, 12976.97it/s]
<ipython-input-12-6c302e787d8f>:11: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  fp_rru = fp_rru.dropna(1)


In [15]:
fp_ru1.head()

,41,80,95,97,110,128,226,247,264,302,...,452,293,844,955,227,496,111,148,208,662
0,1,1,1,1,1,1,2,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,1,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,1,0,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,1,0,2,0,2,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,1,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
fp_ru2.head()

,80,95,97,110,128,226,247,264,411,541,...,729,734,909,923,0,290,894,365,111,662
0,1,1,1,1,1,2,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,1,0,2,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,1,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,1,0,2,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,1,0,2,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
fp_rru.head()

,14,36,90,97,314,356,540,551,650,653,...,962,174,908,365,567,723,905,484,792,959
0,1,1,1,1,1,2,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,1,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,7,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,1,4,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# A list for the evaluation result
rmse_sum = {}
r2_sum = {}

RU1

In [33]:
from sklearn.model_selection import train_test_split

X = fp_ru1.values
y = df["DFT band gap/eV"].values

#Seperating literature data to training and validation data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [34]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

Lasso(alpha=0.1)

In [35]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

R_square_train = r2_score(y_train, lasso.predict(X_train))
RMSE_train = mean_squared_error(y_train, lasso.predict(X_train), squared=False)
R_square_test = r2_score(y_test, lasso.predict(X_test))
RMSE_test = mean_squared_error(y_test, lasso.predict(X_test), squared=False)

print(f'R_square(Train)_ru1: {R_square_train}\nRMSE(Train): {RMSE_train}\n')
print(f'R_square(Test)_ru1: {R_square_test}\nRMSE(Test): {RMSE_test}')

R_square(Train)_ru1: 0.6704856479653383
RMSE(Train): 0.6986976065287736

R_square(Test)_ru1: 0.702102282706964
RMSE(Test): 0.5736909406640696


In [36]:
rmse_sum['RU1'] = [RMSE_train, RMSE_test] 
r2_sum['RU1'] = [R_square_train, R_square_test] 

RU2

In [37]:
from sklearn.model_selection import train_test_split

X = fp_ru2.values
y = df["DFT band gap/eV"].values

#Seperating literature data to training and validation data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [38]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

Lasso(alpha=0.1)

In [39]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

R_square_train = r2_score(y_train, lasso.predict(X_train))
RMSE_train = mean_squared_error(y_train, lasso.predict(X_train), squared=False)
R_square_test = r2_score(y_test, lasso.predict(X_test))
RMSE_test = mean_squared_error(y_test, lasso.predict(X_test), squared=False)

print(f'R_square(Train)_ru2: {R_square_train}\nRMSE(Train): {RMSE_train}\n')
print(f'R_square(Test)_ru2: {R_square_test}\nRMSE(Test): {RMSE_test}')

R_square(Train)_ru2: 0.6878627256272019
RMSE(Train): 0.6800250402908142

R_square(Test)_ru2: 0.6872172026904722
RMSE(Test): 0.5878490678232731


In [40]:
rmse_sum['RU2'] = [RMSE_train, RMSE_test] 
r2_sum['RU2'] = [R_square_train, R_square_test] 

RRU

In [41]:
from sklearn.model_selection import train_test_split

X = fp_rru.values
y = df["DFT band gap/eV"].values

#Seperating literature data to training and validation data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [42]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

Lasso(alpha=0.1)

In [43]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

R_square_train = r2_score(y_train, lasso.predict(X_train))
RMSE_train = mean_squared_error(y_train, lasso.predict(X_train), squared=False)
R_square_test = r2_score(y_test, lasso.predict(X_test))
RMSE_test = mean_squared_error(y_test, lasso.predict(X_test), squared=False)

print(f'R_square(Train)_rru: {R_square_train}\nRMSE(Train): {RMSE_train}\n')
print(f'R_square(Test)_rru: {R_square_test}\nRMSE(Test): {RMSE_test}')

R_square(Train)_rru: 0.7221829740452079
RMSE(Train): 0.6415514884563931

R_square(Test)_rru: 0.7664800309975826
RMSE(Test): 0.507932984602974


In [44]:
rmse_sum['RRU'] = [RMSE_train, RMSE_test] 
r2_sum['RRU'] = [R_square_train, R_square_test] 

Summary

In [48]:
summary = pd.DataFrame(r2_sum, index=["train", "test"])
summary.style.set_caption("Table of R Square")

,RU1,RU2,RRU
train,0.670486,0.687863,0.722183
test,0.702102,0.687217,0.766480


In [47]:
summary = pd.DataFrame(rmse_sum, index=["train", "test"])
summary.style.set_caption("Table of RMSE")

,RU1,RU2,RRU
train,0.698698,0.680025,0.641551
test,0.573691,0.587849,0.507933
